In [1]:
import spacy
import es_dep_news_trf
import pandas as pd
import re
import time

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'


c:\Users\samue\anaconda3\envs\cdas_o22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp = spacy.load("es_dep_news_trf")
nlp = es_dep_news_trf.load()

In [3]:
df_entradas=pd.read_feather('../data/solicitudes.feather') # cargar base#
#df_entradas=pd.read_csv('../data/Prueba 1 ITAM.csv')

In [4]:

columna_interes= "DESCRIPCIONSOLICITUD"
#columna_interes= "Descripción"
num_entradas = 10000
#num_entradas = df_entradas.shape[0]
df_entradas_100 = df_entradas.loc[0:(num_entradas-1), [columna_interes]]

num_palabras = sum([len(texto.split()) for texto in df_entradas_100[columna_interes]])
tamanio_archivo = df_entradas_100.memory_usage(deep=True).sum()


In [5]:
#%% reemplazamos las palabras de regex por #####
def testar_regex(texto):

    #correo electronico
    regex=re.sub("([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)", '#####', texto)

    #clave de elector
    regex=re.sub("[A-Z]{6}[0-9]{6,}(H|M)[0-9]{3}", '#####', regex)

    #curp
    regex=re.sub("[A-Z]{4}[0-9]{6}[A-Z]{6}([A-Z][0-9]|0[0-9])", '#####', regex)

    #RFC
    regex=re.sub("[A-Z,Ñ,&]{3,4}([0-9]{2})(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z|\d]{3}", '#####', regex)

    #numero cartilla militar
    regex=re.sub("(A|B|C)([0-9]{7,8}|\-[0-9]{7,8})", '#####', regex)

    #numero tel
    regex=re.sub("[0-9]{8,}|[0-9]{3} [0-9]{7}|[0-9]{2,3}(\-|.)[0-9]{4}(\-|.)[0-9]{4}", '#####', regex)

    return regex

In [6]:
startTime = time.time() 
df_entradas_100[columna_interes] = df_entradas_100[columna_interes].apply(lambda x: testar_regex(x))

In [7]:
df_entradas_100['entidades'] = df_entradas_100[columna_interes].apply(lambda X: nlp(X))

In [8]:
df_entradas_100['entidades'] = df_entradas_100['entidades'].apply(lambda X: [(entidad.text, entidad.pos_) for entidad in X])


In [9]:
df_entradas_100['testado'] = df_entradas_100['entidades'].apply(lambda x: [pair[0] if pair[1]!="PROPN" else "####" for pair in x])
endTime = time.time()

In [10]:
df_entradas_100.head()

,DESCRIPCIONSOLICITUD,entidades,testado
0,Copia de los documentos y los datos con que cuenta PEP respecto al comportamiento histórico de los pozos típicos comprendidos en el Proyecto ATG estudios científicos y variables que pudieran presentarse con los que PEMEX Exploración y Producción elaboró la evaluación integral del Proyecto ATG presentada ante la SHCP incluyendo en forma específica: 1.- La producción de aceite promedio en barriles de petróleo por pozo al principio de cada uno de los años considerados en la evaluación integral del proyecto ATG; 2.-La declinación anual promedio de petróleo por pozo usada durante el primer y segundo años de producción; 3.- El precio del petróleo y del gas considerados para cada año; 4.- Los responsables de la elaboración de la evaluación integral del Proyecto ATG; 5.- El costo de operación anual promedio de petróleo por pozo para cada uno de los años considerados en la evaluación integral del proyecto ATG.,"[(Copia, NOUN), (de, ADP), (los, DET), (documentos, NOUN), (y, CCONJ), (los, DET), (datos, NOUN), (con, ADP), (que, PRON), (cuenta, VERB), (PEP, PROPN), (respecto, NOUN), (al, DET), (comportamiento, NOUN), (histórico, ADJ), (de, ADP), (los, DET), (pozos, NOUN), (típicos, ADJ), (comprendidos, ADJ), (en, ADP), (el, DET), (Proyecto, PROPN), (ATG, PROPN), ( , SPACE), (estudios, NOUN), (científicos, ADJ), (y, CCONJ), (variables, NOUN), (que, PRON), (pudieran, AUX), (presentarse, VERB), ( , SPACE), (con, ADP), (los, DET), (que, PRON), (PEMEX, PROPN), (Exploración, PROPN), (y, CCONJ), (Producción, PROPN), (elaboró, VERB), (la, DET), (evaluación, NOUN), (integral, ADJ), (del, ADP), (Proyecto, PROPN), (ATG, PROPN), (presentada, ADJ), (ante, ADP), (la, DET), (SHCP, PROPN), ( , SPACE), (incluyendo, VERB), (en, ADP), (forma, NOUN), (específica, ADJ), (:, PUNCT), (1.-, PUNCT), (La, DET), (producción, NOUN), (de, ADP), (aceite, NOUN), (promedio, NOUN), ( , SPACE), (en, ADP), (barriles, NOUN), (de, ADP), (petróleo, NOUN), (por, ADP), (pozo, NOUN), ( , SPACE), (al, ADP), ( , SPACE), (principio, NOUN), (de, ADP), (cada, DET), (uno, PRON), (de, ADP), (los, DET), ( , SPACE), (años, NOUN), (considerados, ADJ), (en, ADP), (la, DET), (evaluación, NOUN), (integral, ADJ), (del, ADP), (proyecto, NOUN), (ATG, PROPN), (;, PUNCT), (2.-La, DET), (declinación, NOUN), (anual, ADJ), (promedio, NOUN), (de, ADP), (petróleo, NOUN), (por, ADP), (pozo, NOUN), ( , SPACE), (usada, ADJ), ...]","[Copia, de, los, documentos, y, los, datos, con, que, cuenta, ####, respecto, al, comportamiento, histórico, de, los, pozos, típicos, comprendidos, en, el, ####, ####, , estudios, científicos, y, variables, que, pudieran, presentarse, , con, los, que, ####, ####, y, ####, elaboró, la, evaluación, integral, del, ####, ####, presentada, ante, la, ####, , incluyendo, en, forma, específica, :, 1.-, La, producción, de, aceite, promedio, , en, barriles, de, petróleo, por, pozo, , al, , principio, de, cada, uno, de, los, , años, considerados, en, la, evaluación, integral, del, proyecto, ####, ;, 2.-La, declinación, anual, promedio, de, petróleo, por, pozo, , usada, ...]"
1,Del ATG para precisar información y datos usados en la tabla de flujo de efectivo (TFE) sobre pozo típico de Agua Fría elaborada para responder a la solicitud con folio ##### solicito: 1.- La producción de aceite en BD al principio de cada uno de los 10 años considerados en la tabla; 2.- El costo del capital usado para que se recuperara la inversión en 7 meses; 3.- El precio del aceite y del gas considerados para cada año; 4.- La fórmula usada o explicación detallada de cómo se realizó la estimación del costo de operación; 5.- La explicación detallada del origen del incremento en la producción de aceite durante los años 5 9 y 10.,"[(Del, ADP), (ATG, PROPN), ( , SPACE), (para, ADP), (precisar, VERB), (información, NOUN), (y, CCONJ), (datos, NOUN), (usados, ADJ), (en, ADP), (la, DET), (tabla, NOUN), (de, ADP), (flujo, NOUN), (de, ADP), (efectivo, NOUN), ((, PUNCT), (TFE, PROPN

In [11]:
print('Total entradas: ' + str(df_entradas_100.shape[0]) + '\nTotal palabras: ' + str(num_palabras) + '\nTotal memoria: ' + str(tamanio_archivo/100) + " KB" + '\nTiempo de corrida: ' + str(int((endTime - startTime)/60)) + ' min ' + str((endTime - startTime)%60) + ' seg')

Total entradas: 10000
Total palabras: 474477
Total memoria: 37774.17 KB
Tiempo de corrida: 30 min 3.796013355255127 seg
